In [1]:

import os
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../../../ukb-dementia-shap/")


from logic.data_processing.data_import import dataload
from logic.data_processing.data_processing import data_proc_main
from logic.analysis.analysis import AnalysisCharts
from logic.ml.classification_shap import IDEARs_funcs
from ukb_utils.utils import basic_funcs
from pandas.tseries.offsets import DateOffset


import pandas as pd
import numpy as np
import re

ac=dataload()
dp=data_proc_main()
ml=IDEARs_funcs()
an=AnalysisCharts()


In [3]:
df=pd.read_parquet(dp.path+'df_ad_20211214.parquet')

In [4]:
df.shape

(468500, 1732)

In [3]:
#dp.studyvars(depvar="AD")

In [6]:
vars_PD=['AST_ALT_ratio',
 'age_when_attended_assessment_centre_f21003_0_0',
 'alanine_aminotransferase_f30620_0_0',
 'apolipoprotein_a_f30630_0_0',
 'avg_duration_to_first_press_of_snapbutton_in_each_round',
 'total_bilirubin_f30840_0_0',
 'creactive_protein_f30710_0_0',
 'chest_pain_or_discomfort_f2335_0_0',
 'cholesterol_f30690_0_0',
 'coffee_intake_f1498_0_0',
 'forced_vital_capacity_fvc_f3062_0_0',
 'glycated_haemoglobin_hba1c_f30750_0_0',
 'hand_grip_strength_left_f46_0_0',
 'hand_grip_strength_right_f47_0_0',
 'hip_circumference_f49_0_0',
 'igf1_f30770_0_0',
 'lymphocyte_count_f30120_0_0',
 'mean_corpuscular_volume_f30040_0_0',
 'mother_still_alive_f1835_0_0',
 'neuroticism_score_f20127_0_0',
 'neutrophill_percentage_f30200_0_0',
 'neutrophill_lymphocyte_ratio',
 'number_of_selfreported_noncancer_illnesses_f135_0_0',
 'number_of_treatmentsmedications_taken_f137_0_0',
 'parental_pd',
 'platelet_count_f30080_0_0',
 'Retired',
 'smoking_status_f20116_0_0',
 'suffer_from_nerves_f2010_0_0',
 'testosterone_f30850_0_0',
 'Total ICD10 Conditions at baseline',
 'triglycerides_f30870_0_0',
 'urate_f30880_0_0',
 'usual_walking_pace_f924_0_0',
 'vitamin_d_f30890_0_0',
 'waist_circumference_f48_0_0','eid','PD']

In [25]:
cols=list(df.columns)
dropwords=ml.wordsremovePD
dropcols=['date_of_attending_assessment_centre_f53_0_0','AD','dementia']
cols=[c for c in cols if c not in dropcols and not re.search(dropwords,c)]
print(len(cols))

1639


In [ ]:
config=dict(scale_pos_weight = 6,subsample = 1, min_child_weight = 5, max_depth = 5, gamma= 2, 
                      colsample_bytree= 0.6,smote=1,reps=2)

In [21]:
mod_xgb=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, learning_rate=0.1,
               max_delta_step=0,  missing=None, 
               n_estimators=60, n_jobs=4, nthread=4, objective='binary:logistic',
               random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=ml.config['scale_pos_weight'],
               min_child_weight=ml.config['min_child_weight'],
               gamma=ml.config['gamma'], colsample_bytree=ml.config['colsample_bytree'],max_depth=ml.config['max_depth'],
               seed=42, silent=None, subsample=1, verbosity=1,eval_metric='auc')


In [27]:
for m in [1,3,5,7]:
    mod_xgb=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, learning_rate=0.1,
               max_delta_step=0,  missing=None, 
               n_estimators=60, n_jobs=4, nthread=4, objective='binary:logistic',
               random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=ml.config['scale_pos_weight'],
               min_child_weight=ml.config['min_child_weight'],
               gamma=ml.config['gamma'], colsample_bytree=ml.config['colsample_bytree'],
            max_depth=m,
               seed=42, silent=None, subsample=1, verbosity=1,eval_metric='auc')
    auc=ml.train_eval(df_train,df_test,model=mod_xgb,dropcols=dropcols,depvar='PD',wordsremove=ml.wordsremovePD,resizeratio=20,shapshow=1)
    print(auc)
    

0.722605582096099
0.7361428629632423
0.7366881649403033
0.7271800853169392


In [28]:
aucs=[]
mcw=[]
max_depth=[]
for m in [1,2,3,4,5,6,7]:
    mod_xgb=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, learning_rate=0.1,
               max_delta_step=0,  missing=None, 
               n_estimators=60, n_jobs=4, nthread=4, objective='binary:logistic',
               random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=ml.config['scale_pos_weight'],
               min_child_weight=m,
               gamma=ml.config['gamma'], colsample_bytree=ml.config['colsample_bytree'],
            max_depth=5,
               seed=42, silent=None, subsample=1, verbosity=1,eval_metric='auc')
    
    mcw.append(m)
    mcw.append(5)
    auc=ml.train_eval(df_train,df_test,model=mod_xgb,dropcols=dropcols,depvar='PD',wordsremove=ml.wordsremovePD,resizeratio=20,shapshow=1)
    aucs.append(round(auc,5))
    print('min child weight '+str(m)+' :'+str(auc))

min child weight 1 :0.7305832360276392
min child weight 2 :0.7306905106793541
min child weight 3 :0.7315438967874819
min child weight 4 :0.7316617818774755
min child weight 5 :0.7366881649403033
min child weight 6 :0.7372950317316254
min child weight 7 :0.7359893496858911


In [30]:
aucs2=[]
posw=[]

for s in [1,6,10,15,20]:
    mod_xgb=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, learning_rate=0.1,
               max_delta_step=0,  missing=None, 
               n_estimators=60, n_jobs=4, nthread=4, objective='binary:logistic',
               random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=s,
               min_child_weight=6,
               gamma=ml.config['gamma'], colsample_bytree=ml.config['colsample_bytree'],
            max_depth=5,
               seed=42, silent=None, subsample=1, verbosity=1,eval_metric='auc')
    
    posw.append(s)
    auc=ml.train_eval(df_train,df_test,model=mod_xgb,dropcols=dropcols,depvar='PD',wordsremove=ml.wordsremovePD,resizeratio=20,shapshow=1)
    aucs.append(round(auc,5))
    print('scale positive weight '+str(s)+' :'+str(auc))


scale positive weight 7 :0.7314854786963377
scale positive weight 7 :0.7372950317316254
scale positive weight 7 :0.7356550948741354
scale positive weight 7 :0.7336246782844031
scale positive weight 7 :0.7339494495835143


In [32]:
aucs3=[]
lr=[]

for lr in [0.05,0.1,0.15,0.2,0.4,0.6]:
    mod_xgb=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, learning_rate=lr,
               max_delta_step=0,  missing=None, 
               n_estimators=60, n_jobs=4, nthread=4, objective='binary:logistic',
               random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=6,
               min_child_weight=6,
               gamma=ml.config['gamma'], colsample_bytree=ml.config['colsample_bytree'],
            max_depth=5,
               seed=42, silent=None, subsample=1, verbosity=1,eval_metric='auc')
    
    posw.append(s)
    auc=ml.train_eval(df_train,df_test,model=mod_xgb,dropcols=dropcols,depvar='PD',wordsremove=ml.wordsremovePD,resizeratio=20,shapshow=1)
    aucs.append(round(auc,5))
    print('scale positive weight '+str(lr)+' :'+str(auc))



scale positive weight 0.05 :0.7249635352870756
scale positive weight 0.1 :0.7372950317316254
scale positive weight 0.15 :0.7284878738467433
scale positive weight 0.2 :0.7192111433874171
scale positive weight 0.4 :0.6895591033486161
scale positive weight 0.6 :0.659630585108458


In [7]:
df=pd.read_parquet(dp.path+'df_PD_final.parquet')

In [19]:
df_train,df_test=ml.holdout_data(df,agemin=50,agemax=70,depvar='PD',apoe=3,holdout_ratio=0.3)

Total PD in data: 1835


In [26]:
ml.train_eval(df_train,df_test,model=mod_xgb,dropcols=dropcols,depvar='PD',wordsremove=ml.wordsremovePD,resizeratio=20,shapshow=1)

ValueError: Length of values (245984) does not match length of index (105421)

In [8]:
print("Dataset has {} entries and {} features".format(*df.shape))

Dataset has 466930 entries and 1750 features


In [15]:

import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
#from sklearn.model_selection import cross_val_score
from sklearn import metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV  #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

train = df[vars_PD]
target = 'PD'
IDcol = 'eid'


    
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):

    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])

    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['PD'],eval_metric='auc')

    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]

    #Print model report:
    print( "\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(dtrain[target].values, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain[target], dtrain_predprob))

    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [16]:
#Choose all predictors except target & IDcols
predictors = [x for x in train.columns if x not in [target, IDcol]]
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, train, predictors)


Model Report
Accuracy : 0.9959
AUC Score (Train): 0.870280


TypeError: 'NoneType' object is not callable

In [18]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4, cv=5)
gsearch1.fit(train[predictors],train[target])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

[18:09:01] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


AttributeError: 'GridSearchCV' object has no attribute 'grid_scores_'

In [ ]:
param_test2 = {
 'max_depth':[4,5,6],
 'min_child_weight':[4,5,6]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(train[predictors],train[target])
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

In [ ]:
'eid' in cols

In [5]:
X, y = df[cols].values, df['PD'].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.1, random_state=42)

In [ ]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
#Choose all predictors except target & IDcols
predictors = [x for x in train.columns if x not in [target, IDcol]]
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, train, predictors)